<a href="https://colab.research.google.com/github/hamdansyaif/DeepLearning/blob/main/Week_2/Compressed/compressed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load dataset
df = pd.read_csv("compressed_data.csv")


# Hapus baris dengan NaN di target
df = df.dropna(subset=['Cover_Type'])

# Pisahkan fitur (X) dan target (y)
X = df.drop(columns=["Cover_Type"])
y = df["Cover_Type"] - 1  # Mengurangi 1 agar kelas mulai dari 0 (karena PyTorch membutuhkan label 0-based)


# Bagi dataset menjadi data latih dan uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# Normalisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Konversi ke Tensor PyTorch
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_train_torch = torch.tensor(y_train.values, dtype=torch.long)
y_test_torch = torch.tensor(y_test.values, dtype=torch.long)

# Definisi Model PyTorch
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


# Inisialisasi model
input_size = X_train.shape[1]
hidden_size = 128
num_classes = len(np.unique(y))
model_torch = MLPModel(input_size, hidden_size, num_classes)


# Definisi loss function dan optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_torch.parameters(), lr=0.001)


# Early stopping manual PyTorch
early_stop_patience = 5
best_loss = float("inf")
patience = 0


# Training model PyTorch
for epoch in range(200):
    optimizer.zero_grad()
    outputs = model_torch(X_train_torch)
    loss = criterion(outputs, y_train_torch)
    loss.backward()
    optimizer.step()

    # Validasi
    with torch.no_grad():
        val_outputs = model_torch(X_test_torch)
        val_loss = criterion(val_outputs, y_test_torch)

    print(f"Epoch {epoch+1}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")

    # Early stopping
    if val_loss.item() < best_loss:
        best_loss = val_loss.item()
        patience = 0
    else:
        patience += 1
    if patience >= early_stop_patience:
        print("Early stopping!")
        break



Epoch 1, Loss: 2.0173988342285156, Val Loss: 1.9747092723846436
Epoch 2, Loss: 1.9737088680267334, Val Loss: 1.9322001934051514
Epoch 3, Loss: 1.9311919212341309, Val Loss: 1.8908838033676147
Epoch 4, Loss: 1.8898746967315674, Val Loss: 1.850767731666565
Epoch 5, Loss: 1.8497684001922607, Val Loss: 1.811845064163208
Epoch 6, Loss: 1.8108543157577515, Val Loss: 1.7740975618362427
Epoch 7, Loss: 1.773113489151001, Val Loss: 1.7374987602233887
Epoch 8, Loss: 1.7365204095840454, Val Loss: 1.7020190954208374
Epoch 9, Loss: 1.701043725013733, Val Loss: 1.6676244735717773
Epoch 10, Loss: 1.6666481494903564, Val Loss: 1.6342802047729492
Epoch 11, Loss: 1.633299469947815, Val Loss: 1.6019519567489624
Epoch 12, Loss: 1.6009632349014282, Val Loss: 1.5706045627593994
Epoch 13, Loss: 1.5696018934249878, Val Loss: 1.540201187133789
Epoch 14, Loss: 1.5391823053359985, Val Loss: 1.5107088088989258
Epoch 15, Loss: 1.5096722841262817, Val Loss: 1.4820971488952637
Epoch 16, Loss: 1.4810411930084229, Val 

In [ ]:
# Evaluasi PyTorch
y_pred_torch = torch.argmax(model_torch(X_test_torch), axis=1).numpy()
print("Akurasi:", accuracy_score(y_test, y_pred_torch))
print("Presisi:", precision_score(y_test, y_pred_torch, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_torch, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred_torch, average='weighted'))